### Replication code 
Sheng, J., Simutin, M., & Zhang, T. (2023). Cheaper is not better: on the ‘superior’performance of high-fee mutual funds. The review of asset pricing studies, 13(2), 375-404.'


### *** I am beyond grateful to my classmate Taisei Noda for his explanations and guidance in importing and understanding these datasets ***

In [1]:
import wrds 

import pandas as pd 
import numpy as np 

### Import Data 

In [2]:
conn = wrds.Connection()

WRDS recommends setting up a .pgpass file.
Created .pgpass file successfully.
You can create this file yourself at any time with the create_pgpass_file() function.
Loading library list...
Done


In [3]:
# conn.list_tables(library='crsp') 
# conn.list_tables(library='tr_mutualfunds') 

WRDS Thomson Reuters Mutual Funds: https://wrds-www.wharton.upenn.edu/pages/get-data/thomson-reuters/ 

WRDS CRSP Mutual Funds: https://wrds-www.wharton.upenn.edu/data-dictionary/crsp_q_mutualfunds/ 

In [5]:
# DATA SOURCE 1.1 - CRSP Mutual Funds (fund_summary2)
# conn.get_table(library='crsp',table='fund_summary2',obs=5).columns 

# takes 1 minute to run 

crspfundsum = conn.raw_sql(
    """
    select crsp_fundno,ticker,caldt,crsp_obj_cd,si_obj_cd,mgmt_cd,fund_name,index_fund_flag,retail_fund,inst_fund,lipper_class_name
    from crsp.fund_summary2 
    where caldt <= '2018-12-31'
    order by crsp_fundno, caldt
    """, 
    date_cols=['caldt']
)
crspfundsum['crsp_fundno'] = crspfundsum['crsp_fundno'].astype(int)     # convert to integer 
crspfundsum['year'] = crspfundsum['caldt'].dt.year      # extract year 

# sort, then drop duplicates fundno and year (keep the last one) 
crspfundsum = crspfundsum.sort_values(by=['crsp_fundno','caldt']).drop_duplicates(subset=['crsp_fundno','year'],keep='last') 

In [15]:
crspfundsum.shape 

(625736, 12)

In [6]:
# DATA SOURCE 1.2 - CRSP Mutual Funds (monthly_tna_ret_nav) 
# takes 1.5 minute to run 

monthly_tna_ret_nav = conn.raw_sql(
    """
    select crsp_fundno,caldt,mret,mtna,mnav
    from crsp.monthly_tna_ret_nav
    where caldt >= '1980-01-01'
    AND caldt <= '2019-12-31'
    order by crsp_fundno, caldt
    """, 
    date_cols=['caldt']
)
monthly_tna_ret_nav['crsp_fundno'] =monthly_tna_ret_nav['crsp_fundno'].astype(int) 
monthly_tna_ret_nav = monthly_tna_ret_nav.dropna()      # drop missing values 
monthly_tna_ret_nav = monthly_tna_ret_nav.sort_values(by=['crsp_fundno', 'caldt']) 

# ym format, then convert to pandas date format 
monthly_tna_ret_nav["ym"] = pd.to_datetime(monthly_tna_ret_nav['caldt'], format='%Y-%m').dt.to_period('M') 

#monthly_tna_ret_nav = monthly_tna_ret_nav.drop(columns=['caldt'])
#monthly_tna_ret_nav['year'] = monthly_tna_ret_nav.ym.dt.year

In [8]:
monthly_tna_ret_nav 

,crsp_fundno,caldt,mret,mtna,mnav,ym
2,1,1997-03-31,-0.011789,12.081,14.72,1997-03
3,1,1997-04-30,0.008417,20.540,14.76,1997-04
4,1,1997-05-30,0.013523,26.157,14.87,1997-05
5,1,1997-06-30,0.012502,34.609,14.97,1997-06
6,1,1997-07-31,0.029325,42.490,15.32,1997-07
...,...,...,...,...,...,...
327624,96422,2019-11-29,0.000000,342.000,10.04,2019-11
327625,96422,2019-12-31,-0.000996,337.400,10.03,2019-12
327744,96492,2019-12-31,0.028134,0.200,10.24,2019-12
327895,96529,2019-12-31,0.044090,310.500,11.13,2019-12


In [7]:
# DATA SOURCE 1.3 - CRSP Mutual Funds (fund_fees) 
# conn.get_table(library='crsp',table='fund_fees',obs=5).columns 

# VERY FAST  

fund_fees = conn.raw_sql(
    """
    select crsp_fundno,begdt,enddt,exp_ratio,actual_12b1,turn_ratio
    from crsp.fund_fees
    order by crsp_fundno, begdt
    """, 
    date_cols=['begdt']
) 

fund_fees['begdt'] = pd.to_datetime(fund_fees['begdt'])     # convert to Pandas dataframe format 
fund_fees = fund_fees.dropna(subset=['begdt'])
fund_fees['enddt'] = pd.to_datetime(fund_fees['enddt'])
fund_fees['enddt'] = fund_fees['enddt'].fillna(pd.Timestamp('2100-01-01'))      # if end date is missing, fill with 2100-01-01 instead of dropping

In [6]:
fund_fees 

,crsp_fundno,begdt,enddt,exp_ratio,actual_12b1,turn_ratio
0,1.0,1997-10-01,1998-09-30,0.0019,0.0000,1.31
1,1.0,1998-10-01,1999-09-30,0.0042,NaN,0.64
2,2.0,1997-10-01,1998-09-30,0.0000,0.0000,0.06
3,2.0,1998-10-01,1999-09-30,0.0000,NaN,0.32
4,3.0,1993-12-31,1994-12-29,0.0156,0.0025,1.46
...,...,...,...,...,...,...
118900,101987.0,2021-11-01,2022-10-31,0.0000,NaN,0.25
118901,101988.0,2021-11-01,2022-10-31,0.0003,NaN,0.36
118902,101989.0,2021-11-01,2022-10-31,0.0000,NaN,0.32
118903,101990.0,2021-11-01,2022-10-31,0.0000,NaN,0.37


"We remove passive funds by examining fund names and index fund indicators.... We then exclude mutual funds that are not U.S. domestic equity funds based on the CRSP style code, Thomson Reuters style code, and Lipper objective name" 

In [8]:
US_equity_funds = crspfundsum.dropna(subset=['crsp_obj_cd'])       # crsp_obj_cd = fund objective 
US_equity_funds = US_equity_funds[US_equity_funds['crsp_obj_cd'].str.startswith('ED')]      # ED is Equity Domestic ... 
US_equity_funds = US_equity_funds[US_equity_funds['index_fund_flag'].isna()]
crsp_fundno_list = US_equity_funds.crsp_fundno.drop_duplicates().to_list()

## MFLink

In [9]:
# take 20s to run 

# crsp_fundno is CRSP Mutual Funds unique identifier 
# wficn is Thomson Reuteres unique identifier 

mflink1 = conn.raw_sql(
    """
    select crsp_fundno,wficn
    from mfl.mflink1
    """, 
)
mflink1[['crsp_fundno','wficn']] = mflink1[['crsp_fundno','wficn']].astype(int)

mflink2 = conn.raw_sql(
    """
    select fundno,wficn,rdate,fdate
    from mfl.mflink2
    """,
    date_cols=['rdate']
)

mflink2['rdate'] = pd.to_datetime(mflink2['rdate'])
mflink2['fdate'] = pd.to_datetime(mflink2['fdate'])
mflink2['ym'] = mflink2['rdate'].dt.to_period('M')  # convert rdate to year-month (ym) 

c:\Users\lkn3\AppData\Local\Programs\Python\Python310\lib\site-packages\wrds\sql.py:580: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  full_df = pd.concat([full_df, chunk])


### Aggregating multiple share classes

In [10]:
### A1 - MERGE US_equity_funds (originally from fund_summary2) with monthly_tna_ret_nav (unique identifier = crsp_fundno) 
fund_monthly_multiclass = monthly_tna_ret_nav.merge(US_equity_funds[['crsp_fundno']].drop_duplicates(),on=['crsp_fundno'])

### A2 - MERGE the output from A1 with MFLINK1 (adding additional attributes for more merges later) (unique identifier = crsp_fundno) 
fund_monthly_multiclass = fund_monthly_multiclass.merge(mflink1,on=['crsp_fundno']) 

### A3 - MERGE the output from A2 with fund_fees (unique identifier = crsp_fundno) 
fund_monthly_multiclass = fund_monthly_multiclass.merge(fund_fees,on=['crsp_fundno']) 

### A4 - Keep if caldt is between begdt and enddt 
fund_monthly_multiclass = fund_monthly_multiclass[(fund_monthly_multiclass['begdt']<=fund_monthly_multiclass['caldt'])&(fund_monthly_multiclass['enddt']>=fund_monthly_multiclass['caldt'])] 
fund_monthly_multiclass = fund_monthly_multiclass.drop(columns=['begdt','enddt'])   # drop the begdt and enddt columns 

### A5 - Aggregate mtna by wficn and ym (gen new column mtna_class_sum = sum of mtna for each class)  
fund_monthly_multiclass = fund_monthly_multiclass.merge(fund_monthly_multiclass.groupby(['wficn','ym'])['mtna'].sum().reset_index(name='mtna_class_sum'),on=['wficn','ym'])

### A6 - Generate the weight of each stock in each fund (weight = mtna/mtna_calss_sum) 
fund_monthly_multiclass['weight'] = fund_monthly_multiclass['mtna']/fund_monthly_multiclass['mtna_class_sum'] 

### A7 - Apply the weight to FUND-LEVEL RETURNS and EXPENSE MEASURES 
fund_monthly_multiclass['mret'] =  fund_monthly_multiclass['mret'] * fund_monthly_multiclass['weight']
fund_monthly_multiclass['exp_ratio'] =  fund_monthly_multiclass['exp_ratio'] * fund_monthly_multiclass['weight']
fund_monthly_multiclass['actual_12b1'] =  fund_monthly_multiclass['actual_12b1'] * fund_monthly_multiclass['weight']
fund_monthly_multiclass['turn_ratio'] =  fund_monthly_multiclass['turn_ratio'] * fund_monthly_multiclass['weight']

### A8 - Collapse by wficn and ym 
# Will look like this in Stata: collapse (sum) mret exp_ratio actual_12b1 turn_ratio, by(wficn ym) 
fund_monthly = fund_monthly_multiclass.groupby(['wficn','ym'])[['mret','exp_ratio','actual_12b1','turn_ratio']].sum().reset_index()

### A9 - Generate monthly GROSS RETURN (= returns + expenses charged) 
fund_monthly['gret'] = fund_monthly['mret'] + (1/12)*fund_monthly['exp_ratio']

## Fama-French Factors

In [16]:
ff = conn.raw_sql(
    """
    select date, mktrf, smb, hml, rmw, cma, rf, year, month,
       umd, dateff
    from ff.fivefactors_monthly
    """
    ,date_cols=['date']
)

# Make sure dataframe index is unique
ff =ff.reset_index(drop=True)
ff["ym"] = pd.to_datetime(ff['date'], format='%Y-%m').dt.to_period('M')     # create ym column 
ff = ff.drop(columns=['date'])

In [5]:
fund_monthly

,wficn,ym,mret,exp_ratio,actual_12b1,turn_ratio,gret
0,100001,1987-12,0.074509,0.000000,0.000000,0.00,0.074509
1,100001,1988-03,-0.042676,0.000000,0.000000,0.00,-0.042676
2,100001,1988-06,0.038961,0.000000,0.000000,0.00,0.038961
3,100001,1988-09,0.038869,0.000000,0.000000,0.00,0.038869
4,100001,1988-12,0.020935,0.000000,0.000000,0.00,0.020935
...,...,...,...,...,...,...,...
923431,612101,2016-08,-0.001850,0.011235,0.000735,0.62,-0.000914
923432,612101,2016-09,-0.000927,0.011214,0.000714,0.62,0.000008
923433,612101,2016-10,-0.017846,0.011106,0.000606,0.62,-0.016921
923434,612101,2016-11,0.055715,0.011143,0.000643,0.62,0.056643


In [6]:
# MERGE the panel output from A9 with FF5 (merge on ym) 

fund_monthly_ret = fund_monthly[['wficn','ym','mret','gret','exp_ratio','actual_12b1', 'turn_ratio']].merge(ff,on=['ym'])
fund_monthly_ret

,wficn,ym,mret,gret,exp_ratio,actual_12b1,turn_ratio,mktrf,smb,hml,rmw,cma,rf,year,month,umd,dateff
0,100001,1987-12,0.074509,0.074509,0.000000,0.000000,0.00,0.0681,0.0006,-0.0443,0.0296,-0.0242,0.0039,1987.0,12.0,0.0587,1987-12-31
1,100001,1988-03,-0.042676,-0.042676,0.000000,0.000000,0.00,-0.0227,0.0625,0.0073,-0.0025,0.0180,0.0044,1988.0,3.0,0.0058,1988-03-31
2,100001,1988-06,0.038961,0.038961,0.000000,0.000000,0.00,0.0479,0.0216,-0.0124,0.0147,-0.0337,0.0049,1988.0,6.0,-0.0293,1988-06-30
3,100001,1988-09,0.038869,0.038869,0.000000,0.000000,0.00,0.0330,-0.0136,-0.0075,0.0176,-0.0061,0.0062,1988.0,9.0,0.0027,1988-09-30
4,100001,1988-12,0.020935,0.020935,0.000000,0.000000,0.00,0.0149,0.0201,-0.0169,0.0069,-0.0038,0.0063,1988.0,12.0,0.0049,1988-12-30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923431,612101,2016-08,-0.001850,-0.000914,0.011235,0.000735,0.62,0.0049,0.0168,0.0318,-0.0185,-0.0034,0.0002,2016.0,8.0,-0.0349,2016-08-31
923432,612101,2016-09,-0.000927,0.000008,0.011214,0.000714,0.62,0.0025,0.0185,-0.0124,-0.0221,0.0003,0.0002,2016.0,9.0,0.0019,2016-09-30
923433,612101,2016-10,-0.017846,-0.016921,0.011106,0.000606,0.62,-0.0202,-0.0402,0.0409,0.0093,0.0028,0.0002,2016.0,10.0,0.0059,2016-10-31
923434,612101,2016-11,0.055715,0.056643,0.011143,0.000643,0.62,0.0486,0.0707,0.0821,-0.0018,0.0370,0.0001,2016.0,11.0,-0.0462,2016-11-30


In [13]:
# B1 - Filter MFLINKS1 to retain observations that appear in crsp_fundno_list (US equity funds) 
crsp_thomson_list = mflink1[mflink1['crsp_fundno'].isin(crsp_fundno_list)] 

# Merge the data output from B1 with MFLINK2 (merge on wficn) 
crsp_thomson_list = crsp_thomson_list.merge(mflink2.drop(columns='fdate'),on=['wficn']) 

thomson_fundno_list = crsp_thomson_list['fundno'].drop_duplicates().to_list()   # drop duplicates 

# Converts the list of Thomson fund numbers into a tuple and stores it in a dictionary under 'funds'   
fundno_list = {'funds':tuple(thomson_fundno_list)} 

Get fund holdings data from Thomson Reuters (s12 master file) 

In [16]:
# SO LONG - takes 8 minutes to run 

tfns12_holdings = conn.raw_sql(
    """
    select fundno, fundname,rdate,fdate,assets
    from tfn.s12
    where fundno in %(funds)s\
    """, 
    date_cols=['rdate'],params=fundno_list
) 

In [17]:
# SO LONG - 8 minutes to run 

tfns12 = conn.raw_sql(
    """
    select fundno,cusip, shares, ticker,fdate
    from tfn.s12
    where fundno in %(funds)s\
    """, 
    date_cols=['rdate'],params=fundno_list
) 

In [18]:
# CONVERT DATE AND EXTRACT PERIODS 
tfns12_holdings['fdate'] = pd.to_datetime(tfns12_holdings['fdate'])
tfns12_holdings['rdate'] = pd.to_datetime(tfns12_holdings['rdate'])

tfns12_holdings['ym'] = tfns12_holdings['rdate'].dt.to_period('M')
tfns12_holdings['year'] = tfns12_holdings['rdate'].dt.year
tfns12['fdate'] = pd.to_datetime(tfns12['fdate'])
#tfns12['rdate'] = pd.to_datetime(tfns12['rdate']) 
#tfns12['ym'] = tfns12['rdate'].dt.to_period('M') 

In [19]:
tfns12_holdings 

,fundno,fundname,rdate,fdate,assets,ym,year
0,76021.0,TSB HIGH INCOME TR,NaT,1997-03-31,NaN,NaT,NaN
1,77938.0,THORNTON LITTLE DRGN TR,NaT,1997-03-31,NaN,NaT,NaN
2,77938.0,THORNTON LITTLE DRGN TR,NaT,1997-06-30,NaN,NaT,NaN
3,73255.0,ADIG AKTIEN FRANKREICH,NaT,1997-09-30,NaN,NaT,NaN
4,76581.0,FCP PARIT.GROUPE DIVERSI,NaT,1997-09-30,NaN,NaT,NaN
...,...,...,...,...,...,...,...
269665,43550.0,NEW PERSPECTIVE FUND,2023-12-31,2023-12-31,12672111.0,2023-12,2023.0
269666,64095.0,GUIDEMARK LARGE CAP VALU,2023-12-31,2023-12-31,4543.0,2023-12,2023.0
269667,84596.0,AST FIRST CAP APPR TARGE,2023-12-31,2023-12-31,1286803.0,2023-12,2023.0
269668,88136.0,AQR TM EMERGING MULTI-ST,2023-12-31,2023-12-31,45472.0,2023-12,2023.0


In [20]:
tfns12

,fundno,cusip,shares,ticker,fdate
0,76021.0,None,NaN,None,1997-03-31
1,77938.0,None,NaN,None,1997-03-31
2,77938.0,None,NaN,None,1997-06-30
3,73255.0,None,NaN,None,1997-09-30
4,76581.0,None,NaN,None,1997-09-30
...,...,...,...,...,...
269665,43550.0,Y9T53H12,2003018.0,VBL,2023-12-31
269666,64095.0,Y9T53H12,11304.0,VBL,2023-12-31
269667,84596.0,Y9T53H12,54330.0,VBL,2023-12-31
269668,88136.0,Y9T53H12,66688.0,VBL,2023-12-31


### Merging

In [21]:
# MERGE US_equity_funds (originally from fund_summary2) with MFLINK1 
crsp_mf_link = US_equity_funds.merge(mflink1,on=['crsp_fundno']) 
crsp_mf_link = crsp_mf_link.drop_duplicates(subset=['wficn','year'])

In [23]:
holdings = tfns12_holdings.sort_values(by=['fundno','rdate']).drop_duplicates(subset=['fundno','rdate'],keep='last')

# M1 - MERGE TR fund holdings with MFLINK2 
holdings = holdings.merge(mflink2.dropna(subset='wficn'),on=['fundno','rdate','fdate']) 

holdings['ym'] = holdings['rdate'].dt.to_period('M') 
holdings = holdings.sort_values(by=['wficn','rdate','assets']).drop_duplicates(subset=['fundno','ym'],keep='last')

In [25]:
# M2 - MERGE CRSP_MKFLINK with TR from M1  
holdings2 = holdings.merge(crsp_mf_link[['wficn','year','mgmt_cd','inst_fund']],on=['wficn','year'])
holdings2 = holdings2.merge(tfns12,on=['fundno','fdate']) 

## Stock-level Characteristics 

In [4]:
# CRSP MONTHLY STOCK RETURNS 

crsp_msenames = conn.raw_sql(
    """
    select ncusip,permno,ticker
    from crsp.msenames
    """, 
) 

# M3 - Then MERGE with CRSP_MFLINKS_TR from M2 
stocks = crsp_msenames[['ncusip','permno']].dropna().merge(holdings2[['wficn','rdate','shares','cusip']],left_on=['ncusip'],right_on=['cusip'])

In [5]:
permno_list = stocks.permno.drop_duplicates().to_list()
permno_list = {'permnos':tuple(permno_list)}

# GET ANOTHER DATA 
crsp_msf = conn.raw_sql(
    """
    select permno,date,prc,cfacshr,shrout,ret
    from crsp.msf 
    where permno in %(permnos)s\
    """,
    date_cols=['date'],params = permno_list
)


crsp_msf['ym'] = crsp_msf['date'].dt.to_period('M')
crsp_msf['prc'] = np.abs(crsp_msf['prc'])   # Take absolute value of prc to make sure prices are non-negative 
crsp_msf = crsp_msf.dropna(subset=['ret'])
crsp_msf['mktcap'] = crsp_msf['prc'] * crsp_msf['shrout']
crsp_msf = crsp_msf.dropna(subset=['ret'])

In [7]:
# MERGE the output with Fama-French Five Factors (merge on ym) 

stock_mret = crsp_msf.merge(ff[['mktrf','rf','ym']],on=['ym'])

### Compustat

In [8]:
# OBTAIN LINKING VARIABLES FROM CRSP TO MERGE WITH COMPUSTAT  
link = conn.raw_sql(
    """
    select distinct gvkey, lpermno as permno, linkdt, linkenddt
    from crsp.Ccmxpf_linktable
    where linktype in ('LU', 'LC')
    and LINKPRIM in ('P', 'C')
    and lpermno in %(permnos)s\
    """,
    params=permno_list
)

# convert strings or floats to ints
link['gvkey'] = link.gvkey.astype(str)
link['permno'] = link.permno.astype(int)

# fill in missing end dates with a future date
link['linkenddt'] = pd.to_datetime(link.linkenddt).fillna(pd.Timestamp('21000101'))
gvkey_list = link['gvkey'].drop_duplicates().reset_index(drop=True)
gvkey_list = {'gvkeys':tuple(gvkey_list.astype(str))} 

### WRDS Compustat Annual Fundamentals: https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-annual/ 

In [18]:
# OBTAIN FIRM-LEVEL CHARACTERISTICS DATA  
compa = conn.raw_sql(
    """
    select gvkey, tic, cusip, datadate, fyr, at, opiti, bkvlps, seq, cshi, ppegt, capx, ch, che, chs, xrd, csho, ceq,txdb,lt,pstk,revt,cogs,xsga,tie
    from comp.funda 
    where datadate >= '1980-01-01' 
    and indfmt='INDL' and datafmt='STD' and popsrc='D' and consol='C'
    and gvkey in %(gvkeys)s\
    order by gvkey, datadate
    """, 
    date_cols=['datadate'],params=gvkey_list
) 

# MERGE COMPUSTAT WITH CRSP 
compa = compa.merge(link, on='gvkey', how='inner') 

In [49]:
# SAVE THESE DATAFRAME USING PICKLE 
# quite long - takes 4 minutes to run 

import os

# Define the base directory where you want to save the DataFrames
base_directory = r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Dataframe (Python Pandas)' 

# Save each DataFrame to the specified directory
crspfundsum.to_pickle(os.path.join(base_directory, 'crspfundsum.pkl'))
monthly_tna_ret_nav.to_pickle(os.path.join(base_directory, 'monthly_tna_ret_nav.pkl'))
fund_fees.to_pickle(os.path.join(base_directory, 'fund_fees.pkl'))
US_equity_funds.to_pickle(os.path.join(base_directory, 'US_equity_funds.pkl'))
mflink1.to_pickle(os.path.join(base_directory, 'mflink1.pkl'))
mflink2.to_pickle(os.path.join(base_directory, 'mflink2.pkl'))
fund_monthly_multiclass.to_pickle(os.path.join(base_directory, 'fund_monthly_multiclass.pkl')) 
fund_monthly.to_pickle(os.path.join(base_directory, 'fund_monthly.pkl')) 
ff.to_pickle(os.path.join(base_directory, 'ff.pkl'))
fund_monthly_ret.to_pickle(os.path.join(base_directory, 'fund_monthly_ret.pkl'))
crsp_thomson_list.to_pickle(os.path.join(base_directory, 'crsp_thomson_list.pkl'))
holdings.to_pickle(os.path.join(base_directory, 'holdings.pkl'))
holdings2.to_pickle(os.path.join(base_directory, 'holdings2.pkl'))
tfns12_holdings.to_pickle(os.path.join(base_directory, 'tfns12_holdings.pkl'))
tfns12.to_pickle(os.path.join(base_directory, 'tfns12.pkl'))
crsp_msenames.to_pickle(os.path.join(base_directory, 'crsp_msenames.pkl'))
stocks.to_pickle(os.path.join(base_directory, 'stocks.pkl'))
crsp_msf.to_pickle(os.path.join(base_directory, 'crsp_msf.pkl'))
link.to_pickle(os.path.join(base_directory, 'link.pkl'))
compa.to_pickle(os.path.join(base_directory, 'compa.pkl')) 
stock_mret.to_pickle(os.path.join(base_directory, 'stock_mret.pkl')) 

In [5]:
# TO OPEN/LOAD THE SAVED DATA BACK TO THE PANDAS DATAFRAME LATER  

compa = pd.read_pickle('compa.pkl')   

In [7]:
folder = r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Data output (STATA)' 

fund_monthly_ret['year'] = fund_monthly_ret['ym'].dt.year
fund_monthly_ret['month'] = fund_monthly_ret['ym'].dt.month 
fund_monthly_ret = fund_monthly_ret.drop(columns='ym') 
fund_monthly_ret = fund_monthly_ret.drop(columns='dateff')
fund_monthly_ret.to_stata(f'{folder}\\fund_monthly_ret.dta')

In [34]:
# EXPORT TO STATA 

# Directory 
folder = r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Data output (STATA)' 

# Export each DataFrame to the specified folder
crspfundsum.to_stata(f'{folder}\\crspfundsum.dta') 

# Need to convert (since pandas datatype variable 'period[M]' not supported in Stata) 
# monthly_tna_ret_nav['ym'] = monthly_tna_ret_nav['ym'].astype(str)        # Convert period[M] to string
monthly_tna_ret_nav['ym'] = monthly_tna_ret_nav['ym'].dt.to_timestamp() 
monthly_tna_ret_nav.to_stata(f'{folder}\\monthly_tna_ret_nav.dta') 

fund_fees.to_stata(f'{folder}\\fund_fees.dta') 
mflink1.to_stata(f'{folder}\\mflink1.dta') 

mflink2['ym'] = mflink2['ym'].dt.to_timestamp() 
mflink2.to_stata(f'{folder}\\mflink2.dta') 

fund_monthly_multiclass['ym'] = fund_monthly_multiclass['ym'].dt.to_timestamp() 
fund_monthly_multiclass.to_stata(f'{folder}\\fund_monthly_multiclass.dta') 

crsp_thomson_list['ym'] = crsp_thomson_list['ym'].dt.to_timestamp() 
crsp_thomson_list.to_stata(f'{folder}\\crsp_thomson_list.dta') 

fund_monthly['year'] = fund_monthly['ym'].dt.year
fund_monthly['month'] = fund_monthly['ym'].dt.month 
fund_monthly = fund_monthly.drop(columns='ym') 
fund_monthly.to_stata(f'{folder}\\fund_monthly.dta') 

US_equity_funds['year'] = US_equity_funds['caldt'].dt.year
US_equity_funds['month'] = US_equity_funds['caldt'].dt.month 
US_equity_funds = US_equity_funds.drop(columns='caldt') 
# Convert all object-type columns to string and handle NaN values
for col in US_equity_funds.select_dtypes(include=['object']).columns:
    US_equity_funds[col] = US_equity_funds[col].fillna('')  # Replace NaNs with empty string
    US_equity_funds[col] = US_equity_funds[col].astype(str)
    US_equity_funds[col] = US_equity_funds[col].apply(lambda x: x[:244])  # Truncate strings to 244 characters
US_equity_funds.to_stata(f'{folder}\\US_equity_funds.dta') 

ff = pd.read_pickle('ff.pkl') 
ff = ff.drop(columns='ym') 
ff = ff.drop(columns='dateff')
ff.to_stata(f'{folder}\\ff.dta') 

fund_monthly_ret['year'] = fund_monthly_ret['ym'].dt.year
fund_monthly_ret['month'] = fund_monthly_ret['ym'].dt.month 
fund_monthly_ret = fund_monthly_ret.drop(columns='ym') 
fund_monthly_ret = fund_monthly_ret.drop(columns='dateff')
fund_monthly_ret.to_stata(f'{folder}\\fund_monthly_ret.dta') 

holdings.to_stata(f'{folder}\\holdings.dta')    # forgot to paste the code, but similar procedure

holdings2['month'] = holdings2['fdate'].dt.month 
holdings2 = holdings2.drop(columns='ym') 
holdings2 = holdings2.drop(columns='ym_y')
holdings2 = holdings2.drop(columns='ym_x') 
holdings2 = holdings2.drop(columns='rdate') 
holdings2 = holdings2.drop(columns='fdate')
holdings2.to_stata(f'{folder}\\holdings2.dta') 

tfns12_holdings['month'] = tfns12_holdings['ym'].dt.month 
tfns12_holdings = tfns12_holdings.dropna(subset='ym') 
tfns12_holdings = tfns12_holdings.drop(columns='ym') 
tfns12_holdings = tfns12_holdings.drop(columns='rdate') 
tfns12_holdings = tfns12_holdings.drop(columns='fdate')
tfns12_holdings.to_stata(f'{folder}\\tfns12_holdings.dta')  # CANNOT EXPORT (latin encode ...) 

compa['year'] = compa['datadate'].dt.year
compa['month'] = compa['datadate'].dt.month 
compa = compa.drop(columns='datadate') 
compa = compa.drop(columns='linkenddt') 
compa = compa.drop(columns='linkdt')
compa.to_stata(f'{folder}\\comp.dta')

stock_mret['year'] = stock_mret['date'].dt.year
stock_mret['month'] = stock_mret['date'].dt.month 
stock_mret = stock_mret.drop(columns='date') 
stock_mret = stock_mret.drop(columns='ym') 
stock_mret.to_stata(f'{folder}\\stock_mret.dta') 


# HAVEN'T EXPORTED YET  (Not important for now)
tfns12.to_stata(f'{folder}\\tfns12.dta')    
crsp_msenames.to_stata(f'{folder}\\crsp_msenames.dta') 
crsp_msf.to_stata(f'{folder}\\crsp_msf.dta')
link.to_stata(f'{folder}\\link.dta') 

stocks.to_stata(f'{folder}\\stocks.dta')    # Cannot export because not enough memories (strange) 

?! 

In [7]:
import pandas as pd 
import os

base_directory = r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Data output (Pandas)'  

file_path = os.path.join(base_directory, 'monthly_tna_ret_nav.pkl')
monthly_tna_ret_nav = pd.read_pickle(file_path) 

file_path = os.path.join(base_directory, 'US_equity_funds.pkl')
US_equity_funds = pd.read_pickle(file_path) 

file_path = os.path.join(base_directory, 'mflink1.pkl')
mflink1 = pd.read_pickle(file_path) 

file_path = os.path.join(base_directory, 'fund_fees.pkl')
fund_fees = pd.read_pickle(file_path)

In [11]:
lipper = US_equity_funds.merge(mflink1,on=['crsp_fundno']) 


In [13]:
folder = r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Data output (STATA)' 

lipper['year'] = lipper['caldt'].dt.year
lipper['month'] = lipper['caldt'].dt.month 
lipper = lipper.drop(columns='caldt') 
# Convert all object-type columns to string and handle NaN values
for col in lipper.select_dtypes(include=['object']).columns:
    lipper[col] = lipper[col].fillna('')  # Replace NaNs with empty string
    lipper[col] = lipper[col].astype(str)
    lipper[col] = lipper[col].apply(lambda x: x[:244])  # Truncate strings to 244 characters
lipper.to_stata(f'{folder}\\lipper.dta') 

In [19]:
folder = r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Dataframe (Stata format)'

compa['year'] = compa['datadate'].dt.year
compa['month'] = compa['datadate'].dt.month 
compa = compa.drop(columns='datadate') 
compa = compa.drop(columns='linkenddt') 
compa = compa.drop(columns='linkdt')

# Convert all object columns to string type
for col in compa.select_dtypes(include=['object']).columns:
    compa[col] = compa[col].astype(str)


compa.to_stata(f'{folder}\\compa.dta') 

### DO THE REST OF FILTERING 

In [1]:
import pandas as pd 
import numpy as np 
import os 

In [25]:
import os

os.chdir(r'C:\Users\lkn3\Desktop\BUSI 525\Replication for Cheaper is not better\Dataframe (Python Pandas)')

## SKIP FROM HERE 

In [17]:
# We eliminate mixed funds or highly levered funds, which hold < 70% or > 130% of their assets in equity
compa = pd.read_pickle('compa.pkl') 

compa['total_equity'] = compa['ceq'] + compa['pstk'] 
compa['equity_ratio'] = (compa['total_equity'] / compa['at']) * 100
compa = compa.dropna(subset=['equity_ratio']) 
compa = compa[(compa['equity_ratio'] >= 70) & (compa['equity_ratio'] <= 130)] 

In [23]:
# We remove extremely small funds (those with < $20 million in asset in 2017, or $6 million in 1980) 

# STRANGE - ALL DATA ARE DELETED AFTER FILTERING 

firm_2018 = compa[(compa['datadate'].dt.year == 2018) & (compa['at'] > 20000000)]
valid_gvkeys = firm_2018['gvkey'].unique() 
compa = compa[compa['gvkey'].isin(valid_gvkeys)] 

compa 

,gvkey,datadate,fyr,at,ceq,txdb,lt,pstk,revt,cogs,xsga,tie,permno,linkdt,linkenddt,total_equity,equity_ratio


In [19]:
# We require a fund to have at least 10 stock holdings

holdings2 = pd.read_pickle('holdings2.pkl')  

In [22]:
holdings_count = holdings2.groupby('fundno').size() 
funds_with_at_least_10_holdings = holdings_count[holdings_count >= 10].index 
holdings2 = holdings2[holdings2['fundno'].isin(funds_with_at_least_10_holdings)]


## REGRESSIONS 

In [143]:
# Clean up memories in Python 

import gc
gc.collect() 

5392

In [ ]:
from IPython import get_ipython
get_ipython().magic('reset -sf') 

In [ ]:
# %load 20230719_LII_script.py
import certifi
import ssl
from urllib.request import build_opener, Request, ProxyHandler, HTTPSHandler
context=ssl.create_default_context(cafile=certifi.where())
https_handler = HTTPSHandler(context=context)
#opener = build_opener(https_handler, ProxyHandler(hcaptcha_settings.PROXIES))
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as ax2
import matplotx
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import matplotlib.pyplot as plt
from statsmodels.formula.api import logit
import seaborn as sns
from pandas.tseries.offsets import MonthEnd

#import getFamaFrenchFactors as gff
from patsy import dmatrix
import warnings
import copy
import numpy as np
import statsmodels.api as sm
from bootstrap_stat import bootstrap_stat as bp
from bootstrap_stat import datasets as d
from statsmodels.discrete.conditional_models import ConditionalLogit
from scipy.stats import norm
import os
warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
#warnings.simplefilter(action='ignore', category=FutureWarning)

from scipy.optimize import minimize
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.regression.rolling import RollingOLS
from statsmodels.regression.rolling import RollingOLS
import itertools
from stargazer.stargazer import Stargazer, LineLocation
from scipy.stats.mstats import winsorize
import wrds
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.float_format', '{:.4f}'.format) 

In [ ]:
returns2 = returns.copy()
returns2['exp_ratio_l1'] = returns2.groupby(['crsp_fundno'])['exp_ratio'].shift(1)
returns2 = returns2.dropna(subset='exp_ratio_l1')
returns2['exp_ratio_d1'] = returns2['exp_ratio'] - returns2['exp_ratio_l1']
returns2['change_exp_ratio'] = (returns2['exp_ratio_d1']!=0).astype(int)
returns2 = returns2.dropna(subset='change_exp_ratio')
returns2 = returns2.merge(returns2.groupby(['crsp_fundno'])['change_exp_ratio'].max().reset_index(name='change_exp_ratio_ever'),on=['crsp_fundno'])
returns2['change_exp_ratio_ever'].describe() 